In [1]:
!module load cuda/11.1

%matplotlib inline
import pathlib
import matplotlib.pyplot as plt
import gin

from typing import Any, Dict, Optional, Sequence

import gin
import jax
from jax import numpy as jnp
from jax import random
from jax.config import config
import numpy as np
import pdb
import imageio

from hypernerf import configs
from hypernerf import datasets
from hypernerf import utils


data_path = pathlib.Path('data/vrig/vrig_choco')
GT_CAM = False

if GT_CAM:
    config_path = pathlib.Path('configs/decompose/train_kubric_gt.gin')
else:
    config_path = pathlib.Path('configs/decompose/train_lf.gin')

gin.parse_config_files_and_bindings(
  config_files=[str(config_path)],
  bindings=f"data_dir='{str(data_path)}'",
  skip_unknown=True)

# Load configurations.
exp_config = configs.ExperimentConfig()
train_config = configs.TrainConfig()
eval_config = configs.EvalConfig()

datasource = exp_config.datasource_cls(
  image_scale=exp_config.image_scale,
  random_seed=exp_config.random_seed,
  # Enable metadata based on model needs.
  use_warp_id=True,
  use_appearance_id=True,
  use_camera_id=True,
  use_time=True,
  mask_interest_region=exp_config.mask_interest_region)


train_eval_ids = utils.strided_subset(
  datasource.train_ids, eval_config.num_train_eval) # returns eval_config.num_train_eval+1 evenly spaced samples 
train_eval_iter = datasource.create_iterator(train_eval_ids, batch_size=0)

# convert all batchs into a list for easy access
train_eval_batches = [ batch for (item_id, batch) in zip(train_eval_ids, train_eval_iter)] 

print(len(train_eval_ids))

# store input gif
rgbs = []
for i in range(len(train_eval_batches)):
    batch = train_eval_batches[i]
    rgbs.append(batch['rgb'])
    
n_img = len(rgbs)
with imageio.get_writer(data_path / 'input.gif', fps= max(n_img // 10, 1), mode='I') as writer:
    for rgb in rgbs:
        writer.append_data(rgb)


2022-04-17 09:35:40.877596: W external/org_tensorflow/tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/software/spack/spack-0.11.2/opt/spack/linux-rhel7-x86_64/gcc-5.4.0/openmpi-1.10.7-jdc7f4fjdq5roxhadufd6h66xkwuytss/lib:/usr/local/software/spack/spack-0.11.2/opt/spack/linux-rhel7-x86_64/gcc-4.8.5/gcc-5.4.0-fis24ggupugiobii56fesif2y3qulpdr/lib64:/usr/local/software/spack/spack-0.11.2/opt/spack/linux-rhel7-x86_64/gcc-4.8.5/gcc-5.4.0-fis24ggupugiobii56fesif2y3qulpdr/lib:/usr/local/Cluster-Apps/cuda/8.0/lib64:/usr/local/Cluster-Apps/cuda/8.0/lib:/usr/local/software/global/lib:/usr/local/Cluster-Apps/vgl/2.5.1/64/lib:/usr/local/software/slurm/current/lib
2022-04-17 09:35:41.052864: W external/org_tensorflow/tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so

171


# store input gif